In [ ]:
import numpy as np
import matplotlib.pyplot as pp
import importlib
import metropolis as met
import time
%matplotlib widget

from matplotlib import animation, rc
from IPython.display import HTML, Image
pp.rcParams.update({
    "text.usetex": True,
})


In [ ]:
importlib.reload(met)

# Test output of `metropolis.py`

In [ ]:
x_dat = np.loadtxt('output.dat')
fig, ax = pp.subplots()

xx = np.linspace(-7,7,1000)
ax.plot(xx, met.f(xx)/np.trapz(met.f(xx),x=xx), 
        color='k', linewidth=2, alpha=0.5,
        label="$P(x)$"

       )

x_dat = np.array(x_dat)
#ax2 = ax.twinx(sharey=True)
bins = np.arange(-7,7.1, 0.25)
ax.hist(x_dat,  bins=bins, 
        density=True,
        histtype='step',
        alpha=1,
        linewidth = 2,
        label=f"N = {Ns[n]}"
       )

# First generate Data

In [ ]:

xmin, xmax = -7, 7
x0 = np.random.uniform(xmin/2, xmax/2)
x_dat1 = [x0]
x_dat2 = [x0]
x_dat3 = [x0]
x_dat_anim = [0]

Ns = [5_00, 5_000, 50_000, 5_000]
for n,x_dat in enumerate((x_dat1, x_dat2, x_dat3, x_dat_anim)):
    N = Ns[n]
    x = x_dat[0]
    for n in range(N):
        x = met.iterate(x)
        x_dat.append(x)

# Plot all histograms with P(x)

In [ ]:

fig, ax = pp.subplots()

xx = np.linspace(xmin, xmax, 1000)
ax.plot(xx, met.f(xx)/np.trapz(met.f(xx),x=xx), 
        color='k', linewidth=2, alpha=0.5,
        label="$P(x)$"

       )
bins = np.arange(xmin, xmax + 0.1, 0.25)

for n,x_dat in enumerate((x_dat1, x_dat2, x_dat3)):
    x_dat = np.array(x_dat)
    #ax2 = ax.twinx(sharey=True)
    
    ax.hist(x_dat,  bins=bins, 
            density=True,
            histtype='step',
            alpha=1,
            linewidth = 2,
            label=f"N = {Ns[n]}"
           )
    
ax.legend(fontsize=12)
ax.set_xlabel("$x$", fontsize=16)
ax.set_ylabel("Density", fontsize=16)
pp.xticks(fontsize=12)
pp.yticks(fontsize=12)

In [ ]:
fig.savefig("../../res/multiple_histograms.png", dpi=600)

# Plot just the P(x)

In [ ]:
fig, ax = pp.subplots()

xx = np.linspace(-7,7,1000)
ax.plot(xx, met.f(xx)/np.trapz(met.f(xx),x=xx), 
        color='k', linewidth=2, alpha=0.5,
       )

ax.set_xlabel("$x$", fontsize=16)
ax.set_ylabel("$P(x)$", fontsize=16)

ax.set_ylim(0,0.5)
pp.xticks(fontsize=12)
pp.yticks(fontsize=12)


In [ ]:
fig.savefig("../../res/plot_of_P.png", dpi=600)

# Animations

## Metropolis Histogram

In [ ]:
def init():
    return (bars,)
    for bar in bars:
        bar.set_height(0)
    return (bars,)

def animate(i):
    x = x_dat[i]
    h, _, = np.histogram(x, bins=bins)
    G.heights += h
    density = heights/norm
    for n,bar in enumerate(bars):
        bar.set_height(density[n])
    
    scatters.set_offsets([x,0])
        
    return (bars,)

class G():
    '''Container for "global" variables'''
    pass

In [ ]:
#x_dat = x_dat_anim
x_dat = np.loadtxt("good_5000pt_sample.dat")

rc('animation', html='html5')


bins = np.arange(xmin, xmax + 0.1, 0.25)
heights, edges = np.histogram(x_dat, bins=bins)
norm = sum(heights)*0.25
widths = np.diff(edges)
centers = edges[:-1] + 0.5*widths


heights = heights*0
G.heights = heights
density = heights/norm




In [ ]:
fig, ax = pp.subplots()

ax.set_xlim((xmin, xmax))
ax.set_ylim((0, 0.5))

ax.set_xlabel("$x$", fontsize=16)
ax.set_ylabel("$P(x)$", fontsize=16)

pp.xticks(fontsize=12)
pp.yticks(fontsize=12)

xx = np.linspace(xmin, xmax, 1000)
ax.plot(xx, met.f(xx)/np.trapz(met.f(xx),x=xx), 
        color='k', linewidth=2, alpha=0.5,
        label="$P(x)$"
       )

bars = ax.bar(centers, density, width = widths)
scatters = ax.scatter([0],[0], s=80, edgecolors=['k'] )

scatters.set_clip_on(False)
ax.xaxis.set_zorder(0)
scatters.set_zorder(10)
#ax.set_axisbelow(True)

In [ ]:
#### SAVE new data if you don't like the old one
#np.savetxt("good_5000pt_sample.dat",x_dat)

In [ ]:
anim = animation.FuncAnimation(
    fig, animate, init_func=init, frames=len(x_dat), interval=0.5, blit=True)

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=60, metadata=dict(artist='K.Shudipto Amin'))

anim.save('../../res/animated_metropolis.mp4', writer=writer)

## Random Walk Step plus histogram

In [ ]:
rc('animation', html='html5')
def animate(i):
    if i > N-hold_count or i==0: 
        # to simulate "freeze" of first and last frame
        return
    if not hasattr(G, 'arrow'):

        G.g = np.random.choice(G.arr)
        
        x_new = x_dat[G.index] + G.g
        
        if x_new < xmin or x_new > xmax:
            # Just to keep the walker bounded
            G.g = -G.g
            x_new = x_dat[G.index] + G.g
        
        x_dat.append(x_new)
        
        
        G.arrow = ax.arrow(
            x_dat[G.index],0, G.g, 0, 
            width=0.1, head_width=0.2, head_length=0.05, length_includes_head=True,
            facecolor = 'r',
            edgecolor = 'k'
        )
        

        G.arrow.set_clip_on(False)
        G.arrow.set_zorder(9)
        
        if not hasattr(G, 'text'):
            G.text = ax.annotate(
                f"$g = {G.g:+5.2f}$",
                xy = (0,4.5),
                ha = 'center',
                fontsize=16,
                color = 'r'
            )
        else:
            G.text.set_text(f"$g = {G.g:+5.2f}$")
        G.index += 1
    else:
        G.arrow.remove()
        
        
        delattr(G, 'arrow')
        
    
        scatters.set_offsets([x_dat[G.index],0])
        G.heights, _, = np.histogram(x_dat, bins=bins)
        #G.heights += h

        for n,bar in enumerate(bars):
            bar.set_height(G.heights[n])
            
        
            
        return None
    

class G():
    '''Container for "global" variables'''
    arr = np.arange(-0.9,0.95,0.01)
    arr = arr[np.abs(arr)>0.21]
    pass

In [ ]:
xmin, xmax = -1.125, 1.125

x_dat = [0]
G.index = 0
hold_count = 5
N = 20 + hold_count

In [ ]:
bins = np.arange(xmin, xmax + 0.1, 0.25)
heights, edges = np.histogram(x_dat, bins=bins)

widths = np.diff(edges)
centers = edges[:-1] + 0.5*widths


#heights = heights*0
G.heights = heights


In [ ]:
fig, ax = pp.subplots()

ax.set_xlim((xmin, xmax))
ax.set_ylim((0, 5))

ax.set_xlabel("$x$", fontsize=16)
ax.set_ylabel("Frequency", fontsize=16)

pp.xticks(fontsize=12)
pp.yticks(fontsize=12)

xx = np.linspace(xmin, xmax, 1000)
'''
ax.plot(xx, met.f(xx)/np.trapz(met.f(xx),x=xx), 
        color='k', linewidth=2, alpha=0.5,
        label="$P(x)$"

       )
'''

bars = ax.bar(centers, G.heights, width = widths)
scatters = ax.scatter([0],[0], s=100, edgecolors=['k'] )

scatters.set_clip_on(False)
ax.xaxis.set_zorder(0)
scatters.set_zorder(10)

In [ ]:

anim = animation.FuncAnimation(
    fig, animate, frames=N, interval=1000, blit=True)

In [ ]:
#anim

In [ ]:
Writer = animation.writers['ffmpeg']
writer = Writer(fps=1, metadata=dict(artist='K.Shudipto Amin'))

anim.save('../../res/animated_random_walk.mp4', writer=writer)